In [1]:
import os
from glob import glob
import json
from pprint import pprint
import random
from collections import Counter
import pandas as pd
from material_parser.core.material_parser import MaterialParserBuilder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
# Load data

df = pd.read_csv('./data/bfo_df_modeling_knn_20220825.csv')
df = df.sample(frac=1).reset_index(drop=True)

# WARNING: EDIT THIS (among other things)
df = df.fillna(0)

features_df = df.drop(['Unnamed: 0', 'recipe_id', 'impurity_code'], axis=1)
labels = df['impurity_code']

feature_list = list(features_df.columns)
print(feature_list)
features = features_df.to_numpy()
labels = labels.to_numpy()

['bi_fe_ratio', 'separate_hydrolysis', 'precursor_concentration', 'pH', 'stirring_time_hr', 'stirring_temp_degC', 'age_days', 'age_temp_degC', 'low_coating_time_sec', 'low_coating_rpm', 'high_coating_time_sec', 'high_coating_rpm', 'dry_time_min', 'dry_degC', 'layer_prebake_time_min', 'layer_prebake_degC', 'layer_annealing_time_min', 'layer_annealing_degC', 'layers', 'final_prebake_time_min', 'final_prebake_degC', 'final_annealing_time_hr', 'final_annealing_degC', 'nitrate_precs', 'air_atm', 'o2_atm', 'n2_atm', 'chem_pca-c1', 'chem_pca-c2', 'chem_pca-c3', 'chem_pca-c4', 'chem_pca-c5', 'chem_pca-c6', 'chem_pca-c7', 'chem_pca-c8', 'chem_pca-c9', 'chem_pca-c10', 'chem_pca-c11', 'chem_pca-c12', 'chem_pca-c13', 'chem_pca-c14', 'chem_pca-c15', 'chem_pca-c16', 'chem_pca-c17', 'chem_pca-c18', 'chem_pca-c19', 'chem_pca-c20', 'chem_pca-c21', 'chem_pca-c22', 'chem_pca-c23', 'chem_pca-c24', 'chem_pca-c25', 'chem_pca-c26', 'chem_pca-c27', 'chem_pca-c28', 'chem_pca-c29', 'chem_pca-c30']


In [3]:
# Load data

lit_df = pd.read_csv('./data/bfo_lit_df_modeling_knn_20220825.csv')
lit_df = lit_df.sample(frac=1).reset_index(drop=True)

# WARNING: EDIT THIS (among other things)
lit_df = lit_df.fillna(0)

lit_features_df = lit_df.drop(['Unnamed: 0', 'recipe_id', 'impurity_code'], axis=1)
lit_labels = lit_df['impurity_code']

lit_feature_list = list(lit_features_df.columns)
print(lit_feature_list)
lit_features = lit_features_df.to_numpy()
lit_labels = lit_labels.to_numpy()

['bi_fe_ratio', 'separate_hydrolysis', 'precursor_concentration', 'pH', 'stirring_time_hr', 'stirring_temp_degC', 'age_days', 'age_temp_degC', 'low_coating_time_sec', 'low_coating_rpm', 'high_coating_time_sec', 'high_coating_rpm', 'dry_time_min', 'dry_degC', 'layer_prebake_time_min', 'layer_prebake_degC', 'layer_annealing_time_min', 'layer_annealing_degC', 'layers', 'final_prebake_time_min', 'final_prebake_degC', 'final_annealing_time_hr', 'final_annealing_degC', 'nitrate_precs', 'air_atm', 'o2_atm', 'n2_atm', 'chem_pca-c1', 'chem_pca-c2', 'chem_pca-c3', 'chem_pca-c4', 'chem_pca-c5', 'chem_pca-c6', 'chem_pca-c7', 'chem_pca-c8', 'chem_pca-c9', 'chem_pca-c10', 'chem_pca-c11', 'chem_pca-c12', 'chem_pca-c13', 'chem_pca-c14', 'chem_pca-c15', 'chem_pca-c16', 'chem_pca-c17', 'chem_pca-c18', 'chem_pca-c19', 'chem_pca-c20', 'chem_pca-c21', 'chem_pca-c22', 'chem_pca-c23', 'chem_pca-c24', 'chem_pca-c25', 'chem_pca-c26', 'chem_pca-c27', 'chem_pca-c28', 'chem_pca-c29', 'chem_pca-c30']


In [4]:
# Load data

suggested_df = pd.read_csv('./data/bfo_only_suggested_df_modeling_knn_20220825.csv')
suggested_df = suggested_df.sample(frac=1).reset_index(drop=True)

# WARNING: EDIT THIS (among other things)
suggested_df = suggested_df.fillna(0)

suggested_features_df = suggested_df.drop(['Unnamed: 0', 'recipe_id', 'impurity_code'], axis=1)
suggested_labels = suggested_df['impurity_code']

suggested_feature_list = list(suggested_features_df.columns)
print(suggested_feature_list)
suggested_features = suggested_features_df.to_numpy()
suggested_labels = suggested_labels.to_numpy()

['bi_fe_ratio', 'separate_hydrolysis', 'precursor_concentration', 'pH', 'stirring_time_hr', 'stirring_temp_degC', 'age_days', 'age_temp_degC', 'low_coating_time_sec', 'low_coating_rpm', 'high_coating_time_sec', 'high_coating_rpm', 'dry_time_min', 'dry_degC', 'layer_prebake_time_min', 'layer_prebake_degC', 'layer_annealing_time_min', 'layer_annealing_degC', 'layers', 'final_prebake_time_min', 'final_prebake_degC', 'final_annealing_time_hr', 'final_annealing_degC', 'nitrate_precs', 'air_atm', 'o2_atm', 'n2_atm', 'chem_pca-c1', 'chem_pca-c2', 'chem_pca-c3', 'chem_pca-c4', 'chem_pca-c5', 'chem_pca-c6', 'chem_pca-c7', 'chem_pca-c8', 'chem_pca-c9', 'chem_pca-c10', 'chem_pca-c11', 'chem_pca-c12', 'chem_pca-c13', 'chem_pca-c14', 'chem_pca-c15', 'chem_pca-c16', 'chem_pca-c17', 'chem_pca-c18', 'chem_pca-c19', 'chem_pca-c20', 'chem_pca-c21', 'chem_pca-c22', 'chem_pca-c23', 'chem_pca-c24', 'chem_pca-c25', 'chem_pca-c26', 'chem_pca-c27', 'chem_pca-c28', 'chem_pca-c29', 'chem_pca-c30']


In [5]:
# Implement SMOTE (Sythetic Minority Oversampling Technique)
X = lit_features_df #df.loc[:, df.columns != 'impurity_code']
y = lit_labels #df.loc[:, df.columns == 'impurity_code']

from imblearn.over_sampling import SMOTE

over_sample = SMOTE(random_state=512)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=512)
columns = X_train.columns

os_data_X, os_data_y = over_sample.fit_resample(X_train, y_train)

#os_data_X = pd.DataFrame(data=os_data_X, columns=columns)

print('length of oversampled data: ', len(os_data_X))
print('number of pure syntheses in oversampled data: ', len(os_data_y[os_data_y==0]))
print('number of impure syntheses in oversampled data: ', len(os_data_y[os_data_y==1]))

length of oversampled data:  410
number of pure syntheses in oversampled data:  205
number of impure syntheses in oversampled data:  205


In [6]:
# Evaluation function

def evaluate(model, test_features, test_labels):
    test_labels = test_labels + 1
    predictions = model.predict(test_features) + 1
    errors = abs(predictions - (test_labels))
    
#     pprint(predictions)
#     pprint(test_labels)
    
    tp = np.count_nonzero(predictions + test_labels == 4)
    tn = np.count_nonzero(predictions + test_labels == 2)
    fp = np.count_nonzero(predictions - test_labels == 1)
    fn = np.count_nonzero(predictions - test_labels == -1)
    
    if tp or fp:
        precision = tp / (tp + fp)
    else:
        precision = 0
        
    if tp or fn:
        recall = tp / (tp + fn)
    else:
        recall = 0
        
    if precision or recall:
        f1 = (2 * precision * recall) / (precision + recall)
    else:
        f1 = 0
    
    mape = 100*np.mean(errors / (test_labels+1))
    accuracy = 100 - mape
#     print('Model Performance')
#     print('Average error: {:0.4f} degrees.'.format(np.mean(errors)))
#     print('Accuracy = {:0.2f}%.'.format(accuracy))
#     print('\n')
#     print("Precision: {:0.4f}".format(precision))
#     print("Recall: {:0.4f}".format(recall))
#     print("F1 score: {:0.4f}".format(f1))
    
    return accuracy, precision, recall, f1

In [7]:
from imblearn.over_sampling import SMOTE


def train_diffs(
    train_features=None, 
    train_labels=None, 
    test_features=None, 
    test_labels=None, 
    criterion='entropy', 
    max_depth=7, #12 
    min_samples_split=4, 
    min_samples_leaf=2, 
    seed=512,
    best_f1=0,
    best_f1_seed=0
):
    # Implement SMOTE (Sythetic Minority Oversampling Technique)
    X = lit_features_df #df.loc[:, df.columns != 'impurity_code']
    y = lit_labels #df.loc[:, df.columns == 'impurity_code']


    over_sample = SMOTE(random_state=seed)
    X_train, test_features, y_train, test_labels = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
    columns = X_train.columns

    os_data_X, os_data_y = over_sample.fit_resample(X_train, y_train)

    lit_train_features = os_data_X
    lit_train_labels = os_data_y
    
#     plus_suggested_train_features = np.concatenate((os_data_X, suggested_features))
#     plus_suggested_train_labels = np.concatenate((os_data_y, suggested_labels))

    plus_suggested_train_features = lit_train_features
    plus_suggested_train_labels = lit_train_labels
    
    lit_tree = DecisionTreeClassifier(
        criterion=criterion,
        splitter="best",
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=seed,
        max_features=None,
    )
    
    plus_suggested_tree = DecisionTreeClassifier(
        criterion=criterion,
        splitter="best",
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=seed,
        max_features=None,
    ) 
    
    lit_tree.fit(lit_train_features, lit_train_labels)
    plus_suggested_tree.fit(plus_suggested_train_features, plus_suggested_train_labels)


    lit_eval = evaluate(lit_tree, test_features, test_labels)
    plus_suggested_eval = evaluate(plus_suggested_tree, test_features, test_labels)
    
    if best_f1 < plus_suggested_eval[3]:
        
        print(plus_suggested_eval[0])
        print(plus_suggested_eval[3])
        
        prev_best_fp = glob('./models/20220825/best_*.pkl')
        
        for fp in prev_best_fp:
            os.remove(fp)
        
        best_f1_seed = seed
        
        # Visualize decision tree

        # Import tools needed for visualization
        from sklearn.tree import export_graphviz
        import pydot
        # Pull out one tree from the forest
        # Export the image to a dot file
        export_graphviz(plus_suggested_tree, out_file = '20220825_best_suggested_tree_lit.dot', feature_names = feature_list, class_names = ['pure', 'impure'], rounded = True, precision = 1)
        # Use dot file to create a graph
        (graph, ) = pydot.graph_from_dot_file('20220825_best_suggested_tree_lit.dot')
        # Write graph to a png file
#         graph.write_png('20220825_best_suggested_tree_lit.png')

        importances = list(plus_suggested_tree.feature_importances_)
        # List of tuples with variable and importance
        feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
        # Sort the feature importances by most important first
        feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
        # Print out the feature and importances 
        #[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];
        
        best_f1 = plus_suggested_eval[3]
        
#         with open(f'./models/20220825/best_{seed}_{best_f1:.2f}.pkl', 'wb') as fp:
#             pickle.dump(plus_suggested_tree, fp)
        


    
    accs = plus_suggested_eval[0] - lit_eval[0]
    precs = plus_suggested_eval[1] - lit_eval[1]
    recs = plus_suggested_eval[2] - lit_eval[2]
    f1s = plus_suggested_eval[3] - lit_eval[3]
    
    return (
        accs, 
        precs, 
        recs, 
        f1s, 
        lit_eval[0], 
        lit_eval[1],
        lit_eval[2],
        lit_eval[3],
        plus_suggested_eval[0], 
        plus_suggested_eval[1], 
        plus_suggested_eval[2],
        plus_suggested_eval[3],
        best_f1,
        best_f1_seed
    )

accuracies = []
precisions = []
recalls = []
f1_scores = []

lit_accs = []
lit_precisions = []
lit_recalls = []
lit_f1s = []

plus_suggested_accs = []
plus_suggested_precisions = []
plus_suggested_recalls = []
plus_suggested_f1s = []

best_f1 = 0
best_f1_seed = 0
for seed_base in range(100):
    diffs = train_diffs(seed=seed_base**2, best_f1=best_f1, best_f1_seed=best_f1_seed)
    accuracies.append(diffs[0])
    precisions.append(diffs[1])
    recalls.append(diffs[2])
    f1_scores.append(diffs[3])
    
    lit_accs.append(diffs[4])
    lit_precisions.append(diffs[5])
    lit_recalls.append(diffs[6])
    lit_f1s.append(diffs[7])
    
    plus_suggested_accs.append(diffs[8])
    plus_suggested_precisions.append(diffs[9])
    plus_suggested_recalls.append(diffs[10])
    plus_suggested_f1s.append(diffs[11])
    
    best_f1 = diffs[-2]
    
    best_f1_seed = diffs[-1]

print('===========')
print(best_f1)
print(np.mean(accuracies))
print(np.mean(f1_scores))

print()
print(np.mean(lit_accs))
print(np.mean(lit_precisions))
print(np.mean(lit_recalls))
print(np.mean(lit_f1s))
print()
print(np.mean(plus_suggested_accs))
print(np.mean(plus_suggested_precisions))
print(np.mean(plus_suggested_recalls))
print(np.mean(plus_suggested_f1s))

print(best_f1_seed)

90.44117647058823
0.65


NameError: name 'prev_best_fp' is not defined

In [16]:
accuracy_improvements = []
f1_improvements = []

for seed in [2**j for j in range(1,100)]:
    print("Training from purely literature")
    print("===============================")
    lit_acc, lit_f1 = train_diffs(lit_features, lit_labels, seed=seed)
    print()
    print("Training with Added Data")
    print("========================")
    acc, f1 = train_diffs(features, labels, seed=seed)
    
    accuracy_improvements.append(acc - lit_acc)
    f1_improvements.append(f1 - lit_f1)
    print('\\\\\\\\\\\\\\\\\\\\\\\\\\\\')
    print('\\\\\\\\\\\\\\\\\\\\\\\\\\\\')

Training from purely literature
93.13725490196079
0.5714285714285715


ValueError: too many values to unpack (expected 2)

In [ ]:
print("average improvements to accuracy: ", np.mean(accuracy_improvements))
print("average improvements to f1: ", np.mean(f1_improvements))

### Grid Search for Hyperparameters

In [ ]:
param_dict = {
    "criterion" : ['gini', 'entropy'],
    "max_depth" : range(2,10),
    "min_samples_split" : range(2,10),
    "min_samples_leaf" : range(2,5)
}

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=0)

grid = GridSearchCV(
    grid_tree,
    param_grid=param_dict,
    cv=10,
    verbose=1,
)

grid.fit(train_features, train_labels)
print(grid.best_estimator_.get_params())

## Miscellaneous

In [ ]:
with open('./models/20220825/best_100_0.71.pkl', 'rb') as fp:
    saved_model = pickle.load(fp)

In [ ]:
for feat_name, imp in zip(feature_list, saved_model.feature_importances_.tolist()):
    print(feat_name, imp)